In [27]:
import pandas as pd

api_bank_df = pd.read_csv('Datasets/API-Bank/cleaned_api_bank_data.csv')

In [28]:
# print(api_bank_df.instruction.iloc[0])
# print(api_bank_df.input.iloc[0])


print(api_bank_df.prompt.iloc[0])


Generate an API request in the format of [ApiName(key1='value1', key2='value2', ...)] based on the previous dialogue context.
The current time is 2039-03-09 18:56:09 Wednesday.
Input: 
User: User's utterence
AI: AI's response

Expected output:
API-Request: [ApiName(key1='value1', key2='value2', ...)]

API descriptions:

{"apiCode": "Get_All_Sessions", "description": "Get the list of all available yoga and meditation sessions.", "parameters": {}, "response": {"data": {"description": "List of available sessions.", "type": "list", "items": {"type": "object", "properties": {"session_name": {"description": "Name of the session.", "type": "string"}, "session_date": {"description": "Date of the session.", "type": "string", "format": "formatted"}, "session_time": {"description": "Time of the session.", "type": "string", "format": "formatted"}, "session_instructor": {"description": "Name of the session instructor.", "type": "string"}, "session_description": {"description": "Description of the 

In [29]:
from datasets import load_dataset, load_metric

In [30]:
from transformers import AutoTokenizer


MODEL_NAME ="openai-community/gpt2"# "google/gemma-2b-it" #"openai-community/gpt2"  #"google/gemma-2b-it"

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

tokeniser = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364


/home/ac-99/miniconda3/envs/tool-learning/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [31]:
rouge = load_metric('rouge')

In [40]:
inputs = tokeniser.encode_plus(
    text="This is an example sentence.",
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    return_attention_mask=False,
)
inputs

{'input_ids': [1212, 318, 281, 1672, 6827, 13]}

In [32]:
# from peft import LoraConfig

api_bank_df['tokenised_prompt'] = api_bank_df['prompt'].apply(lambda x: tokeniser.encode(x))

api_bank_df['tokenised_output'] = api_bank_df['output'].apply(lambda x: tokeniser.encode(x))

Token indices sequence length is longer than the specified maximum sequence length for this model (1114 > 1024). Running this sequence through the model will result in indexing errors


In [33]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:
import torch
from transformers import default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)
